In [1]:
import mysql.connector
from sqlalchemy import create_engine
from config import MYSQL_CONNECTION, MYSQL_CONNECTION2, SQLALCH1, SQLALCH2

In [2]:
db = mysql.connector.connect(**MYSQL_CONNECTION)
cur = db.cursor()

con1 = create_engine(SQLALCH1)

con = create_engine(SQLALCH2)

In [3]:
import pandas as pd
from transliterate import translit
from tqdm.auto import tqdm

## copy

In [4]:
tablenames = [
#     "author_name",
    "authors",
    "author_wd_features",
    "author_wiki_data",
    "author_wiki_properties_labels",
    "geo_production",
    "geo_wiki",
    "images",
    "museums",
    "technologies",
    "technology_name",
    "__tech_raw",
    "typology"
]

In [5]:
df = pd.read_sql_table("author_name", con=SQLALCH1)#.head(1000)

In [8]:
def to_en(x):
    if x is None:
        return None
    return translit(x, language_code="ru", reversed=True)

def to_ru(x):
    if x is None:
        return None
    return translit(x, language_code="ru")

In [7]:
df["name_en"] = df[["name_en", "name_ru"]].apply(
    lambda x: to_en(x["name_ru"]) if x["name_en"] is None else x["name_en"], axis=1
)

In [8]:
df["name_ru"] = df[["name_en", "name_ru"]].apply(
    lambda x: to_en(x["name_en"]) if x["name_ru"] is None else x["name_ru"], axis=1
)

In [9]:
to_ru("Luigi Rossini")

'Луиги Россини'

In [10]:
df[df["name_ru"].isnull()]

,id,name_en,name_ru,wiki_id
29,30,None,None,Q81644873
39,40,None,None,Q20604405
262,263,None,None,Q16432712
310,311,None,None,Q13209494
446,447,None,None,Q20617698
...,...,...,...,...
56220,56221,None,None,Q12131926
56252,56253,None,None,Q12562083
56298,56299,None,None,Q12150387
56300,56301,None,None,Q12087800


In [12]:
df.columns = ["author_id", "name_en", "name_ru", "qid"]

In [14]:
df.to_sql("author_name", index=False, con=con, if_exists="replace")

In [41]:
# from wikidata.client import Client

In [48]:
# def get_en_label(x):
#     try:
#         client = Client()
#         entity = client.get(x, load=True)
#         for lang in ["en", "fr", "it", "de"]:
#             res = entity.label.get(lang)
#             if res:
#                 return res
#         return None
#     except:
#         print(x)
#         return x

# def get_ru_label(x):
#     client = Client()
#     entity = client.get(x, load=True)
#     if entity.label.get("ru"):
#         return entity.label.get("ru")
#     else:
#         candidates = dict(entity.label)
#         print(candidates)
#         return candidates[candidates.keys()[0]]

In [43]:
# tqdm.pandas(desc="")

In [15]:
# df["name_en"] = df[["name_en", "wiki_id"]].progress_apply(
#     lambda x: get_en_label(x["wiki_id"]) if x["name_en"] is None else x["name_en"], axis=1
# )

In [16]:
# df[df["name_ru"].isnull()].shape, df[df["name_en"].isnull()].shape

In [ ]:
# df[["name_en", "wiki_id"]].head(2000).progress_apply(
#     lambda x: get_en_label(x["wiki_id"]) if x["name_en"] is None else x["name_en"], axis=1
# )

In [17]:
# df["name_ru"] = df[["name_en", "name_ru"]].apply(
#     lambda x: to_ru(x["name_en"]) if x["name_ru"] is None else x["name_ru"], axis=1
# )

In [ ]:
# for x in df[df["name_ru"].isnull()]["name_en"]:
#     client.get(x, load=True).label

In [18]:
# client = Client()
# entity = client.get("Q20625509", load=True)

```sql
CREATE TABLE museum_site.collection AS
SELECT 
    collection.id, collection.name, collection.museum_copuk, collection.typology,
    time.startYear, time.startMonth, time.startDay, time.finishYear, time.finishMonth, time.finishDay, 
    time.sure as time_sure, geo_production.qid as geo_id
FROM collection
LEFT JOIN time ON collection.id = time.id
LEFT JOIN geo_production ON collection.id = geo_production.id;
```

In [ ]:
SELECT ROW_NUMBER(), text
FROM (
    GROUP_CONCAT(__author_raw.trim_auth SEPARATOR ", ") as text
    FROM __author_raw
    GROUP BY __author_raw.
) as t

In [1]:
# collection = pd.read_sql_query("""
# SELECT 
#     collection.id, collection.name, collection.museum_copuk, collection.typology,
#     time.startYear, time.startMonth, time.startDay, time.finishYear, time.finishMonth, time.finishDay, 
#     time.sure as time_sure, geo_production.qid as geo_id
# FROM collection
# LEFT JOIN time ON collection.id = time.id
# LEFT JOIN geo_production ON collection.id = geo_production.id
# """, con=con1)

```
CREATE TABLE museum_site.author_raw (id INTEGER PRIMARY KEY auto_increment, text TEXT);
CREATE TABLE museum_site.time_raw (id INTEGER PRIMARY KEY auto_increment, text TEXT);
CREATE TABLE museum_site.technology_raw (id INTEGER PRIMARY KEY auto_increment, text TEXT);
CREATE TABLE museum_site.description_raw (id INTEGER PRIMARY KEY auto_increment, text TEXT);
CREATE TABLE museum_site.provenance_raw (id INTEGER PRIMARY KEY auto_increment, text TEXT);
CREATE TABLE museum_site.production_raw (id INTEGER PRIMARY KEY auto_increment, text TEXT);
CREATE TABLE museum_site.find_raw (id INTEGER PRIMARY KEY auto_increment, text TEXT);
```

In [4]:
with con.connect() as to, con1.raw_connection().cursor() as fro:
    fro.execute("""
    SELECT DISTINCT GROUP_CONCAT(__author_raw.trim_auth SEPARATOR ", ") as text
    FROM __author_raw
    GROUP BY __author_raw.obj_id;""")
    data = fro.fetchall()
    print("yes")
    trans = to.begin()
    to.execute("INSERT INTO author_raw (text) VALUES (%s)", data)
    trans.commit()

yes


In [5]:
with con.connect() as to, con1.raw_connection().cursor() as fro:
    fro.execute("""
    SELECT DISTINCT GROUP_CONCAT(__tech_raw.tech_text SEPARATOR ", ") as text
    FROM __tech_raw
    GROUP BY __tech_raw.obj_id;""")
    data = fro.fetchall()
    print("yes")
    trans = to.begin()
    to.execute("INSERT INTO technology_raw (text) VALUES (%s)", data)
    trans.commit()

yes


In [1]:
# with con.connect() as to, con1.raw_connection().cursor() as fro:
#     fro.execute("""
#     SELECT DISTINCT description as text
#     FROM textual;""")
#     data = fro.fetchall()
#     print("yes")
#     trans = to.begin()
#     to.execute("INSERT INTO description_raw (text) VALUES (%s)", data)
#     trans.commit()

In [4]:
with con.connect() as to, con1.raw_connection().cursor() as fro:
    fro.execute("""
    SELECT DISTINCT productionPlace as text
    FROM geo_obj;""")
    data = fro.fetchall()
    print("yes")
    trans = to.begin()
    to.execute("INSERT INTO production_raw (text) VALUES (%s)", data)
    trans.commit()

yes


In [5]:
with con.connect() as to, con1.raw_connection().cursor() as fro:
    fro.execute("""
    SELECT DISTINCT findPlace as text
    FROM geo_obj;""")
    data = fro.fetchall()
    print("yes")
    trans = to.begin()
    to.execute("INSERT INTO find_raw (text) VALUES (%s)", data)
    trans.commit()

yes


In [6]:
with con.connect() as to, con1.raw_connection().cursor() as fro:
    fro.execute("""
    SELECT DISTINCT periodStr as text
    FROM time;""")
    data = fro.fetchall()
    print("yes")
    trans = to.begin()
    to.execute("INSERT INTO time_raw (text) VALUES (%s)", data)
    trans.commit()

yes


In [7]:
df = pd.read_sql_table("technology_name", con=con1)
df.to_sql("technology_name", index=False, con=con, if_exists="replace")

In [9]:
df = pd.read_sql_table("author_wiki_data", con=con1)
df = df[["person_id", "property_id", "entity_id", "entity_value"]]
df.columns = ["author_id", "property_id", "entity_id", "entity_value"]
df.to_sql("author_wikidata", index=False, con=con, if_exists="replace")

In [10]:
df = pd.read_sql_table("author_wiki_properties_labels", con=con1)
df.columns = ["property_id", "qid", "name_en", "name_ru"]
df.to_sql("wikidata_properties", index=False, con=con, if_exists="replace")

In [11]:
df = pd.read_sql_table("author_wd_features", con=con1)
df.columns = ["entity_id", "name_en", "name_ru"]
df.to_sql("wikidata_entities", index=False, con=con, if_exists="replace")

In [12]:
df = pd.read_sql_table("authors", con=con1)[["obj_id", "auth_id"]]
df.columns = ["id", "author_id"]
df.to_sql("authors", index=False, con=con, if_exists="replace")

In [ ]:
# df = pd.read_sql_query("SELECT obj_id, tech_id FROM technologies", con=con1)
# df.columns = ["id", "tech_id"]
# df.to_sql("technologies", index=False, con=con, if_exists="replace")

CREATE TABLE museum_site.technologies (id INTEGER, tech_id INTEGER);
INSERT INTO museum_site.technologies
SELECT obj_id, tech_id FROM museum_20_09_27.technologies;

In [6]:
df = pd.read_sql_table("museums", con=con1).drop("index", axis=1)
names = [i.lower() for i in df.columns]
names[0] = "museum_copuk"
df.columns = names
df.to_sql("museums", index=False, con=con, if_exists="replace")

In [7]:
df = pd.read_sql_table("geo_wiki", con=con1).drop("index", axis=1)
names = [i.lower() for i in df.columns]
names[0] = "geo_id"
df.columns = names
df.to_sql("geo_wiki", index=False, con=con, if_exists="replace")

In [ ]:
CREATE TABLE museum_site.image_link (id INTEGER, image_id INTEGER);
INSERT INTO museum_site.image_link
SELECT obj_id, image_id FROM images ORDER BY obj_id, image_id;


CREATE TABLE museum_site.faces_painting (face_id INT, image_id INT, top INT, `right`INT, bottom INT, `left` INT);
INSERT INTO museum_site.faces_painting
SELECT id, obj_id, top, `right`, bottom, `left` FROM museum_20_09_27.__i_face ORDER BY id, obj_id;

```sql
SELECT * FROM museum_site.collection;

CREATE TABLE collection (
	id INTEGER PRIMARY KEY, `name` TEXT, museum_copuk INT, typology TINYINT, geo_id INT, 
    author_str INT, time_str INT, technology_str INT, description_str INT, production_str INT, find_str INT,
    startYear INT, startMonth TINYINT, startDay TINYINT, finishYear INT, finishMonth TINYINT, finishDay TINYINT, time_sure TINYINT
);

INSERT INTO museum_site.collection (id, `name`, museum_copuk, typology)
SELECT id, `name`, museum_copuk, typology FROM museum_20_09_27.collection;


UPDATE museum_site.collection as newt
INNER JOIN museum_20_09_27.geo_production as oldt ON newt.id = oldt.id
SET geo_id = qid;

```

In [4]:
db = mysql.connector.connect(**MYSQL_CONNECTION)
cur = db.cursor()

db2 = mysql.connector.connect(**MYSQL_CONNECTION2)
cur2 = db2.cursor()

In [5]:
# cur2.execute("SELECT text, id FROM time_raw")
# mapping = dict(cur2.fetchall())

In [5]:
cur.execute("SELECT id, periodStr FROM time WHERE periodStr IS NOT NULL")
data = cur.fetchall()

In [6]:
result = []
mapping = {}
for idx, value in data:
    if value not in mapping:
        mapping[value] = len(mapping)
    result.append((mapping.get(value), idx))

In [7]:
mapping = [(value, key) for key, value in mapping.items()]

In [8]:
len(set(i[0] for i in mapping)), len(mapping)

(918546, 918546)

In [10]:
K = 10000
for i in tqdm(range(len(mapping) // K + 1)):
    cur2.executemany("INSERT INTO time_raw VALUES (%s, %s)", mapping[i * K: (i + 1) * K])
    db2.commit()

In [11]:
K = 10000
for i in tqdm(range(len(result) // K + 1)):
    cur2.executemany("UPDATE collection SET time_str = %s WHERE id = %s", result[i * K: (i + 1) * K])
    db2.commit()

In [5]:
cur.execute("SELECT id, findPlace FROM geo_obj WHERE findPlace IS NOT NULL")
data = cur.fetchall()


result = []
mapping = {}
for idx, value in data:
    if value not in mapping:
        mapping[value] = len(mapping)
    result.append((mapping.get(value), idx))
    
mapping = [(value, key) for key, value in mapping.items()]

K = 10000
for i in tqdm(range(len(mapping) // K + 1)):
    cur2.executemany("INSERT INTO find_raw VALUES (%s, %s)", mapping[i * K: (i + 1) * K])
    db2.commit()

K = 10000
for i in tqdm(range(len(result) // K + 1)):
    cur2.executemany("UPDATE collection SET find_str = %s WHERE id = %s", result[i * K: (i + 1) * K])
    db2.commit()

In [8]:
del data, result, mapping

In [5]:
cur.execute("SELECT id, productionPlace FROM geo_obj WHERE productionPlace IS NOT NULL")
data = cur.fetchall()


result = []
mapping = {}
for idx, value in data:
    if value not in mapping:
        mapping[value] = len(mapping)
    result.append((mapping.get(value), idx))
    
mapping = [(value, key) for key, value in mapping.items()]

K = 10000
for i in tqdm(range(len(mapping) // K + 1)):
    cur2.executemany("INSERT INTO production_raw VALUES (%s, %s)", mapping[i * K: (i + 1) * K])
    db2.commit()

K = 10000
for i in tqdm(range(len(result) // K + 1)):
    cur2.executemany("UPDATE collection SET production_str = %s WHERE id = %s", result[i * K: (i + 1) * K])
    db2.commit()

In [5]:
cur.execute("""
SELECT obj_id, GROUP_CONCAT(__tech_raw.tech_text SEPARATOR ", ") as text
FROM __tech_raw
GROUP BY __tech_raw.obj_id;
""")
data = cur.fetchall()

In [10]:
result = []
mapping = {}
for idx, value in data:
    if value not in mapping:
        mapping[value] = len(mapping)
    result.append((mapping.get(value), idx))

In [11]:
mapping = [(value, key) for key, value in mapping.items()]
len(result), len(mapping)

(17152279, 1402774)

In [12]:
K = 10000
for i in tqdm(range(len(mapping) // K + 1)):
    cur2.executemany("INSERT INTO technology_raw VALUES (%s, %s)", mapping[i * K: (i + 1) * K])
    db2.commit()

In [13]:
K = 10000
for i in tqdm(range(len(result) // K + 1)):
    cur2.executemany("UPDATE collection SET technology_str = %s WHERE id = %s", result[i * K: (i + 1) * K])
    db2.commit()

In [6]:
cur.execute("""
SELECT obj_id, GROUP_CONCAT(__author_raw.trim_auth SEPARATOR ", ") as text
FROM __author_raw
GROUP BY __author_raw.obj_id;
""")
data = cur.fetchall()

In [7]:
result = []
mapping = {}
for idx, value in data:
    if value not in mapping:
        mapping[value] = len(mapping)
    result.append((mapping.get(value), idx))

In [8]:
mapping = [(value, key) for key, value in mapping.items()]
len(result), len(mapping)

(4430700, 718062)

In [9]:
K = 10000
for i in tqdm(range(len(mapping) // K + 1)):
    cur2.executemany("INSERT INTO author_raw VALUES (%s, %s)", mapping[i * K: (i + 1) * K])
    db2.commit()

In [10]:
K = 100000
for i in tqdm(range(len(result) // K + 1)):
    cur2.executemany("UPDATE collection SET author_str = %s WHERE id = %s", result[i * K: (i + 1) * K])
    db2.commit()

In [5]:
cur.execute("SELECT startYear, startMonth, startDay, finishYear, finishMonth, finishDay, sure, id FROM time WHERE id > 14396904 AND (startYear IS NOT NULL OR finishYear IS NOT NULL)")
data = cur.fetchall()

In [7]:
len(data)

9268361

In [23]:
K = 10000
for i in tqdm(range(305, len(data) // K + 1)):
    subres = []
    for row in data[i * K: (i + 1) * K]:
        y, m, d, y2, m2, d2, s, idx = row
        if (m is not None and m > 12) or (m2 is not None and m2 > 12):
            print(row)
        else:
            subres.append(row)
    cur2.executemany("""
    UPDATE collection SET 
    startYear = %s, startMonth = %s, startDay = %s, 
    finishYear = %s, finishMonth = %s, finishDay = %s, time_sure = %s
    WHERE id = %s""", subres)
    db2.commit()

(1850, 500, 1, 1900, 12, 31, 3, 17884058)
(1985, 100, 1, 1985, 100, 1, 3, 22621297)
(1932, 100, 1, 1940, 12, 31, 3, 22986740)
(1914, 100, 1, 1917, 12, 31, 3, 24674734)



In [20]:
# data[i * K: (i + 1) * K]

In [ ]:
# for row in data[i * K: (i + 1) * K]:
#     y, m, d, y2, m2, d2, s, idx = row
# #     print(row)
# #     break
#     if m is None or m > 12 or m < 1 or m2 is None or m2 > 12 or m2 < 1:
#         print(y, m, d, y2, m2, d2, s, idx)

In [ ]:
# cur2.executemany("""
# UPDATE collection SET 
# startYear = %s, startMonth = %s, startDay = %s, 
# finishYear = %s, finishMonth = %s, finishDay = %s, time_sure = %s
# WHERE id = %s
# """, data)
# db2.commit()

In [6]:
df = pd.read_sql_query("SELECT * FROM author_name", con=db2)

In [9]:
ru = set("абвгдежзийклмнопрстуфхцчшщъыьэюя")
en = set("abcdefghijklmnopqrstuvwxyz")

def to_ru(x):
    if x is None:
        return None
    let = set(x)
    
    if "," in x:
        new_x = x.replace(",", "")
    else:
        new_x = " ".join([x.split()[-1]] + x.split()[:-1])

    if len(let & ru) > len(let & en):
        return new_x
    else:
        return translit(new_x, language_code="ru")

In [12]:
df["order_name"] = df["name_ru"].apply(to_ru)

In [19]:
cur2.executemany("UPDATE author_name SET order_name = %s WHERE author_id = %s", df[["order_name", "author_id"]].values.tolist())
db2.commit()

In [20]:
df = pd.read_sql_query("""
SELECT count(authors.id) as n, author_name.author_id
FROM author_name
JOIN authors ON author_name.author_id = authors.author_id
GROUP BY author_name.author_id
ORDER BY author_name.author_id;""", con=db2)

In [22]:
cur2.executemany("UPDATE author_name SET n = %s WHERE author_id = %s", df.values.tolist())
db2.commit()

In [59]:
df = pd.read_sql_query("""
SELECT name_0, name_1, sum(cnt) as cnt FROM geo_wiki 
WHERE name_0 IS NOT NULL
GROUP BY name_0, name_1
ORDER BY name_0, name_1;""", con=db2)

In [60]:
df.head()

,name_0,name_1,cnt
0,Afghanistan,None,4334.0
1,Afghanistan,Badakhshan,7.0
2,Afghanistan,Badghis,2.0
3,Afghanistan,Baghlan,39.0
4,Afghanistan,Balkh,816.0


In [5]:
from collections import defaultdict
import json

In [62]:
data = defaultdict(list)

for country, region, cnt in df.values.tolist():
    if region:
        data[country].append(region)
    else:
        data[country] = []

In [57]:
data

defaultdict(list,
            {'Afghanistan': ['Badakhshan',
              'Badghis',
              'Baghlan',
              'Balkh',
              'Bamyan',
              'Daykundi',
              'Farah',
              'Faryab',
              'Ghazni',
              'Ghor',
              'Hilmand',
              'Hirat',
              'Jawzjan',
              'Kabul',
              'Kandahar',
              'Kapisa',
              'Kunar',
              'Kunduz',
              'Nangarhar',
              'Nimroz',
              'Nuristan',
              'Paktya',
              'Panjshir',
              'Parwan',
              'Sari Pul',
              'Takhar',
              'Wardak',
              'Zabul'],
             'Åland': ['Finström', 'Mariehamn'],
             'Albania': ['Berat',
              'Dibër',
              'Durrës',
              'Fier',
              'Korçë',
              'Lezhë',
              'Shkodër',
              'Tiranë',
              'Vlorë'],
          

In [63]:
data = dict(data)

In [64]:
json.dump(data, open("../museum_app/static/geo_obj.json", "w"), indent=4, ensure_ascii=False)

In [65]:
json.dump(data, open("../museum_app/static/js/geo_obj.js", "w"), ensure_ascii=False)

In [5]:
df = pd.read_sql_query("SELECT * FROM geo_wiki", con=db2)

In [6]:
df.shape, df.drop_duplicates().shape

((39226, 13), (39220, 13))

In [10]:
df = df.drop_duplicates()

In [13]:
df.to_sql("geo_wiki2", con=con, if_exists="replace", index=False)

In [8]:
df = pd.read_sql_query("""
SELECT museums.nl_name_1, museums.nl_name_2, museums.name
FROM museums
LEFT JOIN collection ON collection.museum_copuk = museums.museum_copuk
GROUP BY museums.museum_copuk, museums.nl_name_1, museums.nl_name_2, museums.name
ORDER BY museums.nl_name_1, museums.nl_name_2, count(collection.id) DESC""", con=db2)

In [9]:
data = defaultdict(lambda: defaultdict(list))

for region, district, name in df.values.tolist():
    data[region][district].append(name)

In [11]:
# data

In [13]:
json.dump(data, open("../museum_app/static/geo_museum.json", "w"), indent=4, ensure_ascii=False)
json.dump(data, open("../museum_app/static/geo_museum_2.json", "w"), ensure_ascii=False)

In [5]:
import sqlite3

photodb = sqlite3.connect("/home/dkbrz/github/museum_catalogue/images/faces/photos.db")
photocur = photodb.cursor()

In [6]:
photocur.execute("SELECT id as face_id, obj_id as image_id FROM faces")
data = photocur.fetchall()

In [7]:
len(data)

4281199

In [9]:
data[:10]

[(1, 1168),
 (2, 1293),
 (3, 1301),
 (4, 1332),
 (5, 1332),
 (6, 1348),
 (7, 1348),
 (8, 1478),
 (9, 1654),
 (10, 1817)]

In [ ]:
CREATE TABLE faces_photo (face_id INT, image_id INT);

In [12]:
db2 = mysql.connector.connect(**MYSQL_CONNECTION2)
cur2 = db2.cursor()

In [13]:
# cur2.executemany("INSERT INTO faces_photo VALUES (%s, %s)", data)
# db2.commit()

K = 10000
for i in tqdm(range(len(data) // K + 1)):
    cur2.executemany("INSERT INTO faces_photo VALUES (%s, %s)", data[i * K: (i + 1) * K])
    db2.commit()